# Advent of Code 2021

In [73]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import queue

import common

## Day 9: Smoke Basin

### Part 1:

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Each number corresponds to the height of a particular location, where `9` is the highest and `0` is the lowest a location can be.

Your first goal is to find the **low points** - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are **four** low points, all highlighted: two are in the first row (a `1` and a `0`), one is in the third row (a `5`), and one is in the bottom row (also a `5`). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The **risk level** of a low point is **1 plus its height**. In the above example, the risk levels of the low points are `2`, `1`, `6`, and `6`. The sum of the risk levels of all low points in the heightmap is therefore `15`.

Find all of the low points on your heightmap. **What is the sum of the risk levels of all low points on your heightmap?**

In [112]:
with open("./private/2021/09.in") as f:
	heightmap = [[int(i) for i in line.strip()] for line in f]
heightmap = np.pad(np.array(heightmap), 1, constant_values=10)
heightmap.shape

(102, 102)

In [113]:
rows, cols = [], []
for i in range(1, heightmap.shape[0] - 1):
	for j in range(1, heightmap.shape[1] - 1):
		indices = i+np.array([-1, 0, 0, 1]), j+np.array([0, -1, 1, 0])
		if np.all(heightmap[i,j] < heightmap[indices]):
			rows.append(i)
			cols.append(j)
rows, cols = np.array([rows, cols])

In [114]:
rows, cols

(array([  1,   1,   1,   1,   2,   2,   2,   2,   3,   3,   4,   4,   4,
          4,   5,   5,   5,   6,   6,   7,   7,   7,   7,   8,   8,  10,
         10,  11,  11,  11,  11,  11,  12,  12,  13,  13,  13,  14,  15,
         15,  16,  16,  16,  17,  18,  18,  19,  19,  19,  20,  20,  23,
         23,  23,  23,  25,  25,  25,  25,  25,  26,  26,  26,  27,  28,
         28,  29,  29,  30,  31,  31,  32,  32,  32,  33,  34,  35,  35,
         35,  35,  35,  36,  36,  37,  37,  38,  39,  39,  40,  41,  42,
         42,  42,  43,  43,  44,  44,  45,  45,  45,  46,  46,  46,  48,
         48,  48,  50,  50,  50,  50,  50,  51,  51,  51,  51,  52,  52,
         55,  55,  55,  56,  56,  57,  57,  57,  57,  58,  58,  59,  59,
         59,  59,  60,  60,  60,  60,  61,  63,  63,  64,  64,  65,  65,
         65,  65,  66,  66,  67,  68,  69,  69,  70,  70,  71,  72,  72,
         73,  74,  74,  74,  74,  75,  75,  76,  76,  76,  77,  78,  78,
         79,  79,  80,  80,  80,  81,  81,  82,  82

In [115]:
risk = np.sum(heightmap[rows,cols]+1)
risk

486

### Part 2:

Next, you need to find the largest basins so you know what areas are most important to avoid.

A **basin** is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height `9` do not count as being in any basin, and all other locations will always be part of exactly one basin.

The **size** of a basin is the number of locations within the basin, including the low point. The example above has four basins.

The top-left basin, size `3`:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The top-right basin, size `9`:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The middle basin, size `14`:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

The bottom-right basin, size `9`:

```
2199943210
3987894921
9856789892
8767896789
9899965678
```

Find the three largest basins and multiply their sizes together. In the above example, this is `9 * 14 * 9 = **1134`.

**What do you get if you multiply together the sizes of the three largest basins?**

In [116]:
q = queue.Queue()

sizes = []
for i in range(len(rows)):
	q.put((rows[i], cols[i]))

	search_map = heightmap.copy()
	while not q.empty():
		i,j = q.get()
		search_map[i,j] = 11
		for index in [(i-1, j), (i, j-1), (i, j+1), (i+1, j)]:
			if search_map[index] < 9:
				q.put(index)
	sizes.append(np.sum(search_map == 11))
sizes = np.array(sizes)

In [117]:
np.product(sizes[np.argpartition(sizes, -3)[-3:]])

1059300